## Oleh:
- Putu Gde Kenzie Carlen Mataram (71230994)
<br>
- Tara Tirtanata (71231056)

## 1. Library Imports

Cell ini memuat seluruh library yang digunakan untuk pengambilan data, analisis teknikal, machine learning, backtesting, dan visualisasi.


In [29]:
#cell 1: Library Downloads and Imports
%pip install backtesting yfinance pandas_ta plotly scikit-learn numpy pandas openpyxl

Note: you may need to restart the kernel to use updated packages.


In [30]:
import yfinance as yf
import pandas as pd
import numpy as np
import pandas_ta as ta
from sklearn.ensemble import RandomForestClassifier
from backtesting import Backtest, Strategy
import plotly.graph_objects as go
import plotly.express as px
import warnings

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
warnings.filterwarnings('ignore')

print("Library imports complete.")

Library imports complete.


## 2. Configuration Settings

Cell ini mendefinisikan parameter global untuk proses tuning dan backtesting.

- **Assets:** Daftar aset yang digunakan untuk tuning strategi  
- **Timeframes:** Timeframe intraday hingga hourly  
- **Capital & Leverage:** Modal awal dan leverage simulasi  
- **Parameter Grid:** Ruang pencarian hyperparameter strategi  
- **Ranking Colors:** Kode warna untuk visualisasi peringkat hasil


In [31]:
# cell 2: Configuration Settings

# 1. Aset Tunning
TUNING_TICKERS = ['QQQ', 'NVDA', 'TSLA', 'AAPL', 'AMZN', 'MSFT', 'GOOGL', 'NFLX'] 

# 2. Supported Timeframes
TUNING_TIMEFRAMES = ['1m', '5m', '15m', '30m', '1h']

# 3. Capital & Leverage Settings
CASH = 10_000
LEVERAGE = 1.0 

# 4. Hyperparameter Search Grid
PARAM_GRID = {
    'atr_sl_multiplier': [1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5],       # Jarak Stop Loss
    'adx_filter': [15, 20, 25, 30],                       # Sensitivitas Trend
    'max_layers': [3, 4, 5, 6]               # Agresivitas Pyramiding
}

# 5. Ranking Color Codes (Top 1 → Top 5)
RANK_COLORS = ['#00FF00', '#00BFFF', '#FF4500', '#FFD700', '#C0C0C0'] 


# CONFIGURATION SUMMARY
print(f"Konfigurasi Siap: {len(TUNING_TICKERS)} Aset x {len(TUNING_TIMEFRAMES)} Timeframe.")
print(f"Total Kombinasi per Aset: {len(PARAM_GRID['atr_sl_multiplier']) * len(PARAM_GRID['adx_filter']) * len(PARAM_GRID['max_layers'])} variasi.")

Konfigurasi Siap: 8 Aset x 5 Timeframe.
Total Kombinasi per Aset: 112 variasi.


## 3. Helper Functions

Menyediakan fungsi untuk pengambilan data market dari Yahoo Finance dan penambahan indikator teknikal.

- `get_data()`  
  Mengunduh data OHLCV sesuai batasan timeframe dan menstandarkan timezone ke UTC.

- `add_technical_features()`  
  Menambahkan indikator ADX, EMA-50, ATR, dan RSI untuk analisis trend, volatilitas, dan momentum.


In [32]:
# CELL 3: HELPER FUNCTIONS


def get_data(symbol, interval):
    """
    Mengambil data sesuai batasan Yahoo Finance.
    - 1m: Max 7 hari
    - 5m - 30m: Max 60 hari
    - 1h: Max 730 hari (2 tahun)
    """
    if interval == '1m':
        period = '7d'
    elif interval in ['5m', '15m', '30m']:
        period = '60d'
    elif interval == '1h':
        period = '730d' # Data jam-jaman bisa sampai 2 tahun
    else:
        period = '1y' # Default
    
    try:
        df = yf.download(symbol, period=period, interval=interval, progress=False)
        if isinstance(df.columns, pd.MultiIndex): df.columns = df.columns.droplevel(1)
        df = df.rename(columns={'Adj Close': 'Close'})
        df = df[['Open', 'High', 'Low', 'Close', 'Volume']]
        df = df[df['Volume'] > 0]
        
        # Standarisasi Timezone ke UTC
        if df.index.tz is None: 
            df.index = df.index.tz_localize('UTC')
        else:
            df.index = df.index.tz_convert('UTC')
            
        return df
    except Exception as e:
        print(f"Gagal download {symbol} ({interval}): {e}")
        return None

def add_technical_features(df):
    df = df.copy()
    # Trend Filter (ADX)
    adx = ta.adx(df['High'], df['Low'], df['Close'], length=14)
    df['ADX'] = adx[adx.columns[0]] 
    
    # Trend Direction (EMA)
    df['EMA_50'] = ta.ema(df['Close'], length=50)
    
    # Volatility (ATR)
    df['ATR'] = ta.atr(df['High'], df['Low'], df['Close'], length=14)
    
    # Momentum (RSI) - Opsional, digunakan di ML
    df['RSI'] = ta.rsi(df['Close'], length=14)
    
    return df.dropna()

print("Helper functions defined.")

Helper functions defined.


## 4. Machine Learning Engine

Cell ini melatih model **Random Forest** berbasis data timeframe tinggi (1H) untuk menentukan **bias makro jangka panjang** (bullish / bearish).

Model menghasilkan sinyal `Macro_Signal` yang digunakan sebagai filter arah pada strategi utama.


In [33]:
# CELL 4: MACHINE LEARNING ENGINE

def train_macro_model(symbol):
    """
    Melatih model Random Forest pada data Harian/Jam 
    untuk menentukan bias jangka panjang (Bullish/Bearish).
    """
    # Download data 1 tahun (H1)
    df = yf.download(symbol, period="1y", interval="1h", progress=False)
    
    if isinstance(df.columns, pd.MultiIndex): df.columns = df.columns.droplevel(1)
    df = df.rename(columns={'Adj Close': 'Close'})
    
    # Feature Engineering
    df['RSI'] = ta.rsi(df['Close'], length=14)
    df['Return'] = df['Close'].pct_change()
    df['Vol_Change'] = (df['Volume'] + 1e-9).pct_change() # Anti-Zero Error
    
    # Cleaning Error (Infinity/NaN)
    df = df.replace([np.inf, -np.inf], np.nan)
    df.dropna(inplace=True)
    
    if len(df) < 50: raise ValueError("Data ML tidak cukup.")

    # Target: Close besok > Close hari ini
    df['Target'] = (df['Close'].shift(-1) > df['Close']).astype(int)
    
    features = ['RSI', 'Return', 'Vol_Change']
    X = df[features]
    y = df['Target']
    
    # Training
    model = RandomForestClassifier(n_estimators=100, min_samples_split=10, random_state=42)
    model.fit(X, y)
    
    # Prediksi
    df['Macro_Signal'] = model.predict(X)
    
    if df.index.tz is None: df.index = df.index.tz_localize('UTC')
    else: df.index = df.index.tz_convert('UTC')
        
    return df[['Macro_Signal']]

print("Model ML trained.")

Model ML trained.


## 5. Trading Strategy – Pyramiding

Cell ini mendefinisikan strategi trading berbasis **trend-following** dengan mekanisme **pyramiding**.

Strategi menggunakan:
- **ADX** sebagai filter kekuatan trend  
- **EMA-50** sebagai penentu arah trend  
- **ATR** untuk trailing stop dan risk management  
- **Macro_Signal (ML)** sebagai filter bias jangka panjang  

Posisi ditambah secara bertahap saat trend menguat dan ditutup menggunakan trailing stop atau saat trend patah.


In [34]:
# CELL 5: STRATEGI TRADING (PYRAMID)

class PyramidStrategy(Strategy):
    # Parameter Default (Akan di-overwrite oleh Optimizer)
    atr_sl_multiplier = 2.0   
    adx_filter = 25       
    max_layers = 4            
    
    def init(self):
        self.highest_price = 0
        self.stop_price = 0
    
    def next(self):
        price = self.data.Close[-1]
        atr = self.data.ATR[-1]
        adx = self.data.ADX[-1]
        macro_signal = self.data.Macro_Signal[-1]
        ema_50 = self.data.EMA_50[-1]
        
        # --- 1. EXIT LOGIC (Trailing Stop) ---
        if self.position.is_long:
            if price > self.highest_price:
                self.highest_price = price
                new_stop = self.highest_price - (atr * self.atr_sl_multiplier)
                self.stop_price = max(self.stop_price, new_stop)
            
            if price < self.stop_price:
                self.position.close()
            
            # Hard Exit (Trend Patah)
            if price < ema_50:
                self.position.close()

        # --- 2. ENTRY LOGIC ---
        is_trending = adx > self.adx_filter
        is_uptrend = price > ema_50
        is_macro_bullish = macro_signal == 1
        
        if is_trending and is_uptrend and is_macro_bullish:
            if not self.position:
                self.buy(size=0.30)
                self.highest_price = price
                self.stop_price = price - (atr * self.atr_sl_multiplier)
            
            elif len(self.trades) < self.max_layers:
                last_entry = self.trades[-1].entry_price
                if price > last_entry + (1.0 * atr):
                    self.buy(size=0.20)

print("Pyramid Strategy defined.")

Pyramid Strategy defined.


## 6. Execution Engine – Tuning & Reporting

Cell ini menjalankan proses **tuning parameter secara masif** untuk seluruh kombinasi aset dan timeframe.

Proses utama:
- Melatih **macro ML model** per aset
- Menyiapkan data dan fitur teknikal
- Menjalankan backtest baseline
- Melakukan **grid search optimization**
- Menyimpan metrik performa dan kurva ekuitas terbaik

Output digunakan untuk analisis performa, perbandingan strategi, dan visualisasi hasil tuning.


In [35]:
# CELL 6: EXECUTION ENGINE (TUNING & REPORTING)

tuning_results = []
equity_data_store = {}

print("=" * 70)
print("STARTING OPTIMIZATION PROCESS (MASSIVE PARAMETER TUNING)")
print("=" * 70)

for ticker in TUNING_TICKERS:
    print(f"\nPreparing macro model for asset: {ticker}")
    try:
        macro_brain = train_macro_model(ticker)
    except Exception as e:
        print(f"SKIPPED {ticker} | Macro model training failed: {e}")
        continue

    for tf in TUNING_TIMEFRAMES:
        print(f"Running tuning for {ticker} [{tf}] ...", end=" ")

        try:
            # A. Data Preparation
            df = get_data(ticker, tf)
            if df is None or len(df) < 100:
                print("FAILED (insufficient data).")
                continue

            df = add_technical_features(df)
            df['Macro_Signal'] = macro_brain['Macro_Signal'].reindex(
                df.index, method='ffill'
            )
            df.dropna(inplace=True)

            # B. Baseline Backtest
            bt = Backtest(
                df,
                PyramidStrategy,
                cash=CASH,
                commission=0.0005,
                margin=LEVERAGE
            )
            default_stats = bt.run()

            # C. Parameter Optimization
            best_stats, _ = bt.optimize(
                atr_sl_multiplier=PARAM_GRID['atr_sl_multiplier'],
                adx_filter=PARAM_GRID['adx_filter'],
                max_layers=PARAM_GRID['max_layers'],
                maximize='Return [%]',
                return_heatmap=True
            )

            # D. Store Results
            res = {
                'Ticker': ticker,
                'Timeframe': tf,

                # Core Performance Metrics
                'Return [%]': best_stats['Return [%]'],
                'Buy & Hold Return [%]': best_stats['Buy & Hold Return [%]'],
                'Return (Ann.) [%]': best_stats['Return (Ann.) [%]'],
                'Baseline Return [%]': default_stats['Return [%]'],
                'Return Improvement [%]': (
                    best_stats['Return [%]'] - default_stats['Return [%]']
                ),

                # Trade Statistics
                'Win Rate [%]': best_stats['Win Rate [%]'],
                'Profit Factor': best_stats['Profit Factor'],
                'Max Drawdown [%]': best_stats['Max. Drawdown [%]'],
                'Number of Trades': best_stats['# Trades'],

                # Optimal Parameters
                'ATR SL Multiplier': best_stats._strategy.atr_sl_multiplier,
                'ADX Filter': best_stats._strategy.adx_filter,
                'Max Position Layers': best_stats._strategy.max_layers
            }
            tuning_results.append(res)

            # E. Store Equity Curve
            key = f"{ticker}_{tf}"
            equity_data_store[key] = best_stats._equity_curve['Equity']

            improvement = (
                best_stats['Return [%]'] - default_stats['Return [%]']
            )

            print(
                f"SUCCESS | Return: "
                f"{default_stats['Return [%]']:.2f}% → "
                f"{best_stats['Return [%]']:.2f}% "
                f"(Δ {improvement:.2f}%)"
            )

        except Exception as e:
            print(f"ERROR | {ticker} [{tf}] | {e}")

print("\nOptimization process completed successfully.")

STARTING OPTIMIZATION PROCESS (MASSIVE PARAMETER TUNING)

Preparing macro model for asset: QQQ
Running tuning for QQQ [1m] ... 

Backtest.run:   0%|          | 0/2632 [00:00<?, ?bar/s]

Backtest.optimize:   0%|          | 0/112 [00:00<?, ?it/s]

Backtest.run:   0%|          | 0/2632 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/2632 [00:00<?, ?bar/s]

...

Backtest.run:   0%|          | 0/1735 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/1735 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/1735 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/1735 [00:00<?, ?bar/s]

SUCCESS | Return: 2.32% → 10.82% (Δ 8.50%)

Optimization process completed successfully.


## 7. Final Report – Excel & Visualization (Global Top 5)

Cell ini menghasilkan **laporan akhir hasil tuning global** dan visualisasi performa terbaik.

Proses utama:
- Menggabungkan seluruh hasil tuning ke dalam DataFrame
- Melakukan **ranking global berdasarkan Return**
- Menyimpan laporan lengkap ke file **Excel**
- Menampilkan **Top 5 kombinasi terbaik** (lintas aset & timeframe)
- Memvisualisasikan kurva ekuitas Top 5 secara ter-normalisasi

Output digunakan sebagai dasar evaluasi akhir dan pemilihan strategi terbaik.


In [36]:
# ==============================================================================
# CELL 7: FINAL REPORTING (EXCEL & GLOBAL TOP 5 VISUALIZATION)
# ==============================================================================

if tuning_results:

    # --------------------------------------------------------------------------
    # 1. Build Final DataFrame
    # --------------------------------------------------------------------------
    df_tuning = pd.DataFrame(tuning_results)

    # Global ranking by highest return
    df_tuning.sort_values(by='Return [%]', ascending=False, inplace=True)

    # Column ordering
    cols_order = [
        'Ticker',
        'Timeframe',
        'Return [%]',
        'Buy & Hold Return [%]',
        'Return (Ann.) [%]',
        'Number of Trades',
        'Win Rate [%]',
        'Profit Factor',
        'Max Drawdown [%]',
        'ATR SL Multiplier',
        'ADX Filter',
        'Max Position Layers'
    ]

    final_cols = cols_order + [c for c in df_tuning.columns if c not in cols_order]
    df_tuning = df_tuning[final_cols]

    # --------------------------------------------------------------------------
    # 2. Export Excel Report
    # --------------------------------------------------------------------------
    file_name = "Laporan_Tuning.xlsx"
    df_tuning.to_excel(file_name, index=False)

    print("=" * 70)
    print(f"EXCEL REPORT SAVED: {file_name}")
    print("=" * 70)

    display(
        df_tuning[
            ['Ticker', 'Timeframe', 'Return [%]',
             'Buy & Hold Return [%]', 'Return (Ann.) [%]']
        ].head(5)
    )

    # --------------------------------------------------------------------------
    # 3. Global Top 5 Equity Curve Visualization
    # --------------------------------------------------------------------------
    print("\n" + "=" * 70)
    print("GLOBAL TOP 5 PERFORMANCE VISUALIZATION")
    print("=" * 70)

    top_5_global = df_tuning.head(5)

    if top_5_global.empty:
        print("No data available for visualization.")

    else:
        fig = go.Figure()
        rank = 0

        for _, row in top_5_global.iterrows():
            ticker = row['Ticker']
            timeframe = row['Timeframe']
            key = f"{ticker}_{timeframe}"

            if key not in equity_data_store:
                continue

            equity_curve = equity_data_store[key]

            # Normalize equity curve (start from 0%)
            start_equity = equity_curve.iloc[0]
            normalized_curve = (equity_curve - start_equity) / start_equity * 100

            color = RANK_COLORS[rank % len(RANK_COLORS)]
            label = (
                f"#{rank + 1} {ticker} [{timeframe}] "
                f"({row['Return [%]']:.1f}%)"
            )

            fig.add_trace(
                go.Scatter(
                    x=normalized_curve.index,
                    y=normalized_curve,
                    mode='lines',
                    name=label,
                    line=dict(width=2, color=color)
                )
            )

            rank += 1

        fig.update_layout(
            title=(
                "<b>GLOBAL TOP 5 PERFORMANCE (OPTIMIZED)</b><br>"
                "<sup>Best combinations across all assets and timeframes</sup>"
            ),
            xaxis_title="Time",
            yaxis_title="Profit (%)",
            template="plotly_dark",
            hovermode="x unified",
            height=600,
            legend=dict(
                title="Global Rank",
                yanchor="top",
                y=0.99,
                xanchor="left",
                x=0.01
            )
        )

        fig.show()

else:
    print("No tuning results available.")


EXCEL REPORT SAVED: Laporan_Tuning.xlsx


,Ticker,Timeframe,Return [%],Buy & Hold Return [%],Return (Ann.) [%]
34,GOOGL,1h,19.180969,51.757126,19.348391
9,NVDA,1h,16.791731,26.373381,17.432323
14,TSLA,1h,11.575313,-2.724649,11.673122
39,NFLX,1h,10.817208,3.565115,10.908303
19,AAPL,1h,8.505965,7.318797,9.055906



GLOBAL TOP 5 PERFORMANCE VISUALIZATION
